# Word2Vec Tutorial

In case you missed the buzz, word2vec is a widely featured as a member of the “new wave” of machine learning algorithms based on neural networks, commonly referred to as "deep learning" (though word2vec itself is rather shallow). Using large amounts of unannotated plain text, word2vec learns relationships between words automatically. The output are vectors, one vector per word, with remarkable linear relationships that allow us to do things like vec(“king”) – vec(“man”) + vec(“woman”) =~ vec(“queen”), or vec(“Montreal Canadiens”) – vec(“Montreal”) + vec(“Toronto”) resembles the vector for “Toronto Maple Leafs”.

Word2vec is very useful in [automatic text tagging](https://github.com/RaRe-Technologies/movie-plots-by-genre), recommender systems and machine translation.

Check out an [online word2vec demo](http://radimrehurek.com/2014/02/word2vec-tutorial/#app) where you can try this vector algebra for yourself. That demo runs `word2vec` on the Google News dataset, of **about 100 billion words**.

## This tutorial

In this tutorial you will learn how to train and evaluate word2vec models on your business data.


## Preparing the Input
Starting from the beginning, gensim’s `word2vec` expects a sequence of sentences as its input. Each sentence is a list of words (utf8 strings):

In [1]:
# import modules & set up logging
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
sentences = [['first', 'sentence'], ['second', 'sentence']]
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=1)

2017-12-19 17:52:07,111 : INFO : collecting all words and their counts
2017-12-19 17:52:07,113 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-19 17:52:07,114 : INFO : collected 3 word types from a corpus of 4 raw words and 2 sentences
2017-12-19 17:52:07,115 : INFO : Loading a fresh vocabulary
2017-12-19 17:52:07,116 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2017-12-19 17:52:07,117 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2017-12-19 17:52:07,118 : INFO : deleting the raw counts dictionary of 3 items
2017-12-19 17:52:07,119 : INFO : sample=0.001 downsamples 3 most-common words
2017-12-19 17:52:07,120 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2017-12-19 17:52:07,121 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2017-12-19 17:52:07,122 : INFO : resetting layer weights
2017-12-19 17:52:07,123 : INFO : training model with 3 workers o

Keeping the input as a Python built-in list is convenient, but can use up a lot of RAM when the input is large.

Gensim only requires that the input must provide sentences sequentially, when iterated over. No need to keep everything in RAM: we can provide one sentence, process it, forget it, load another sentence…

For example, if our input is strewn across several files on disk, with one sentence per line, then instead of loading everything into an in-memory list, we can process the input file by file, line by line:

In [3]:
# create some toy data to use with the following example
import smart_open, os

if not os.path.exists('./data/'):
    os.makedirs('./data/')

filenames = ['./data/f1.txt', './data/f2.txt']

for i, fname in enumerate(filenames):
    with smart_open.smart_open(fname, 'w') as fout:
        for line in sentences[i]:
            fout.write(line + '\n')

In [4]:
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()

In [5]:
sentences = MySentences('./data/') # a memory-friendly iterator
print(list(sentences))

[['second'], ['sentence'], ['first'], ['sentence']]


In [6]:
# generate the Word2Vec model
model = gensim.models.Word2Vec(sentences, min_count=1)

2017-12-19 17:52:07,161 : INFO : collecting all words and their counts
2017-12-19 17:52:07,162 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-19 17:52:07,163 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2017-12-19 17:52:07,164 : INFO : Loading a fresh vocabulary
2017-12-19 17:52:07,165 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2017-12-19 17:52:07,166 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2017-12-19 17:52:07,167 : INFO : deleting the raw counts dictionary of 3 items
2017-12-19 17:52:07,169 : INFO : sample=0.001 downsamples 3 most-common words
2017-12-19 17:52:07,170 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2017-12-19 17:52:07,171 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2017-12-19 17:52:07,172 : INFO : resetting layer weights
2017-12-19 17:52:07,173 : INFO : training model with 3 workers o

In [7]:
print(model)
print(model.wv.vocab)

Word2Vec(vocab=3, size=100, alpha=0.025)
{'second': <gensim.models.keyedvectors.Vocab object at 0x7f058032fc90>, 'sentence': <gensim.models.keyedvectors.Vocab object at 0x7f058032f850>, 'first': <gensim.models.keyedvectors.Vocab object at 0x7f058032f9d0>}


Say we want to further preprocess the words from the files — convert to unicode, lowercase, remove numbers, extract named entities… All of this can be done inside the `MySentences` iterator and `word2vec` doesn’t need to know. All that is required is that the input yields one sentence (list of utf8 words) after another.

**Note to advanced users:** calling `Word2Vec(sentences, iter=1)` will run **two** passes over the sentences iterator. In general it runs `iter+1` passes. By the way, the default value is `iter=5` to comply with Google's word2vec in C language. 
  1. The first pass collects words and their frequencies to build an internal dictionary tree structure. 
  2. The second pass trains the neural model.

These two passes can also be initiated manually, in case your input stream is non-repeatable (you can only afford one pass), and you’re able to initialize the vocabulary some other way:

In [8]:
# build the same model, making the 2 steps explicit
new_model = gensim.models.Word2Vec(min_count=1)  # an empty model, no training
new_model.build_vocab(sentences)                 # can be a non-repeatable, 1-pass generator     
new_model.train(sentences, total_examples=new_model.corpus_count, epochs=new_model.iter)                       
# can be a non-repeatable, 1-pass generator

2017-12-19 17:52:07,193 : INFO : collecting all words and their counts
2017-12-19 17:52:07,195 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-19 17:52:07,197 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2017-12-19 17:52:07,198 : INFO : Loading a fresh vocabulary
2017-12-19 17:52:07,199 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2017-12-19 17:52:07,200 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2017-12-19 17:52:07,201 : INFO : deleting the raw counts dictionary of 3 items
2017-12-19 17:52:07,202 : INFO : sample=0.001 downsamples 3 most-common words
2017-12-19 17:52:07,203 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2017-12-19 17:52:07,204 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2017-12-19 17:52:07,205 : INFO : resetting layer weights
2017-12-19 17:52:07,206 : INFO : training model with 3 workers o

0

In [9]:
print(new_model)
print(model.wv.vocab)

Word2Vec(vocab=3, size=100, alpha=0.025)
{'second': <gensim.models.keyedvectors.Vocab object at 0x7f058032fc90>, 'sentence': <gensim.models.keyedvectors.Vocab object at 0x7f058032f850>, 'first': <gensim.models.keyedvectors.Vocab object at 0x7f058032f9d0>}


### More data would be nice
For the following examples, we'll use the [Lee Corpus](https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/test/test_data/lee_background.cor) (which you already have if you've installed gensim):

In [10]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data']) + os.sep
lee_train_file = test_data_dir + 'lee_background.cor'

In [11]:
class MyText(object):
    def __iter__(self):
        for line in open(lee_train_file):
            # assume there's one document per line, tokens separated by whitespace
            yield line.lower().split()

sentences = MyText()

print(sentences)

## Training
`Word2Vec` accepts several parameters that affect both training speed and quality.

### min_count
`min_count` is for pruning the internal dictionary. Words that appear only once or twice in a billion-word corpus are probably uninteresting typos and garbage. In addition, there’s not enough data to make any meaningful training on those words, so it’s best to ignore them:

In [12]:
# default value of min_count=5
model = gensim.models.Word2Vec(sentences, min_count=10)

2017-12-19 17:52:07,247 : INFO : collecting all words and their counts
2017-12-19 17:52:07,248 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-19 17:52:07,265 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-12-19 17:52:07,266 : INFO : Loading a fresh vocabulary
2017-12-19 17:52:07,272 : INFO : min_count=10 retains 806 unique words (7% of original 10186, drops 9380)
2017-12-19 17:52:07,273 : INFO : min_count=10 leaves 40964 word corpus (68% of original 59890, drops 18926)
2017-12-19 17:52:07,277 : INFO : deleting the raw counts dictionary of 10186 items
2017-12-19 17:52:07,279 : INFO : sample=0.001 downsamples 54 most-common words
2017-12-19 17:52:07,280 : INFO : downsampling leaves estimated 26224 word corpus (64.0% of prior 40964)
2017-12-19 17:52:07,281 : INFO : estimated required memory for 806 words and 100 dimensions: 1047800 bytes
2017-12-19 17:52:07,284 : INFO : resetting layer weights
2017-12-19 17:5

### size
`size` is the number of dimensions (N) of the N-dimensional space that gensim Word2Vec maps the words onto.

Bigger size values require more training data, but can lead to better (more accurate) models. Reasonable values are in the tens to hundreds.

In [13]:
# default value of size=100
model = gensim.models.Word2Vec(sentences, size=200)

2017-12-19 17:52:07,423 : INFO : collecting all words and their counts
2017-12-19 17:52:07,424 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-19 17:52:07,439 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-12-19 17:52:07,440 : INFO : Loading a fresh vocabulary
2017-12-19 17:52:07,448 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2017-12-19 17:52:07,449 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2017-12-19 17:52:07,456 : INFO : deleting the raw counts dictionary of 10186 items
2017-12-19 17:52:07,457 : INFO : sample=0.001 downsamples 49 most-common words
2017-12-19 17:52:07,457 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2017-12-19 17:52:07,458 : INFO : estimated required memory for 1723 words and 200 dimensions: 3618300 bytes
2017-12-19 17:52:07,463 : INFO : resetting layer weights
2017-12-19 17:

### workers
`workers`, the last of the major parameters (full list [here](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)) is for training parallelization, to speed up training:

In [14]:
# default value of workers=3 (tutorial says 1...)
model = gensim.models.Word2Vec(sentences, workers=4)

2017-12-19 17:52:07,705 : INFO : collecting all words and their counts
2017-12-19 17:52:07,706 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-19 17:52:07,722 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-12-19 17:52:07,723 : INFO : Loading a fresh vocabulary
2017-12-19 17:52:07,774 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2017-12-19 17:52:07,775 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2017-12-19 17:52:07,781 : INFO : deleting the raw counts dictionary of 10186 items
2017-12-19 17:52:07,783 : INFO : sample=0.001 downsamples 49 most-common words
2017-12-19 17:52:07,784 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2017-12-19 17:52:07,785 : INFO : estimated required memory for 1723 words and 100 dimensions: 2239900 bytes
2017-12-19 17:52:07,790 : INFO : resetting layer weights
2017-12-19 17:

The `workers` parameter only has an effect if you have [Cython](http://cython.org/) installed. Without Cython, you’ll only be able to use one core because of the [GIL](https://wiki.python.org/moin/GlobalInterpreterLock) (and `word2vec` training will be [miserably slow](http://rare-technologies.com/word2vec-in-python-part-two-optimizing/)).

## Memory
At its core, `word2vec` model parameters are stored as matrices (NumPy arrays). Each array is **#vocabulary** (controlled by min_count parameter) times **#size** (size parameter) of floats (single precision aka 4 bytes).

Three such matrices are held in RAM (work is underway to reduce that number to two, or even one). So if your input contains 100,000 unique words, and you asked for layer `size=200`, the model will require approx. `100,000*200*4*3 bytes = ~229MB`.

There’s a little extra memory needed for storing the vocabulary tree (100,000 words would take a few megabytes), but unless your words are extremely loooong strings, memory footprint will be dominated by the three matrices above.

## Evaluating
`Word2Vec` training is an unsupervised task, there’s no good way to objectively evaluate the result. Evaluation depends on your end application.

Google has released their testing set of about 20,000 syntactic and semantic test examples, following the “A is to B as C is to D” task. It is provided in the 'datasets' folder.

For example a syntactic analogy of comparative type is bad:worse;good:?. There are total of 9 types of syntactic comparisons in the dataset like plural nouns and nouns of opposite meaning.

The semantic questions contain five types of semantic analogies, such as capital cities (Paris:France;Tokyo:?) or family members (brother:sister;dad:?). 

Gensim supports the same evaluation set, in exactly the same format:

In [15]:
model.accuracy('./datasets/questions-words.txt')

2017-12-19 17:52:08,009 : INFO : precomputing L2-norms of word weight vectors
2017-12-19 17:52:08,026 : INFO : family: 0.0% (0/2)
2017-12-19 17:52:08,050 : INFO : gram3-comparative: 0.0% (0/12)
2017-12-19 17:52:08,061 : INFO : gram4-superlative: 0.0% (0/12)
2017-12-19 17:52:08,073 : INFO : gram5-present-participle: 5.0% (1/20)
2017-12-19 17:52:08,095 : INFO : gram6-nationality-adjective: 0.0% (0/20)
2017-12-19 17:52:08,123 : INFO : gram7-past-tense: 0.0% (0/20)
2017-12-19 17:52:08,136 : INFO : gram8-plural: 0.0% (0/12)
2017-12-19 17:52:08,145 : INFO : total: 1.0% (1/98)


[{'correct': [], 'incorrect': [], 'section': u'capital-common-countries'},
 {'correct': [], 'incorrect': [], 'section': u'capital-world'},
 {'correct': [], 'incorrect': [], 'section': u'currency'},
 {'correct': [], 'incorrect': [], 'section': u'city-in-state'},
 {'correct': [],
  'incorrect': [(u'HE', u'SHE', u'HIS', u'HER'),
   (u'HIS', u'HER', u'HE', u'SHE')],
  'section': u'family'},
 {'correct': [], 'incorrect': [], 'section': u'gram1-adjective-to-adverb'},
 {'correct': [], 'incorrect': [], 'section': u'gram2-opposite'},
 {'correct': [],
  'incorrect': [(u'GOOD', u'BETTER', u'GREAT', u'GREATER'),
   (u'GOOD', u'BETTER', u'LONG', u'LONGER'),
   (u'GOOD', u'BETTER', u'LOW', u'LOWER'),
   (u'GREAT', u'GREATER', u'LONG', u'LONGER'),
   (u'GREAT', u'GREATER', u'LOW', u'LOWER'),
   (u'GREAT', u'GREATER', u'GOOD', u'BETTER'),
   (u'LONG', u'LONGER', u'LOW', u'LOWER'),
   (u'LONG', u'LONGER', u'GOOD', u'BETTER'),
   (u'LONG', u'LONGER', u'GREAT', u'GREATER'),
   (u'LOW', u'LOWER', u'GOOD',

This `accuracy` takes an 
[optional parameter](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.accuracy) `restrict_vocab` 
which limits which test examples are to be considered.



In the December 2016 release of Gensim we added a better way to evaluate semantic similarity.

By default it uses an academic dataset WS-353 but one can create a dataset specific to your business based on it. It contains word pairs together with human-assigned similarity judgments. It measures the relatedness or co-occurrence of two words. For example, 'coast' and 'shore' are very similar as they appear in the same context. At the same time 'clothes' and 'closet' are less similar because they are related but not interchangeable.

In [16]:
model.evaluate_word_pairs(test_data_dir + 'wordsim353.tsv')

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `evaluate_word_pairs` (Method will be removed in 4.0.0, use self.wv.evaluate_word_pairs() instead).
  """Entry point for launching an IPython kernel.
2017-12-19 17:52:08,199 : INFO : Pearson correlation coefficient against /usr/local/lib/python2.7/dist-packages/gensim-3.2.0-py2.7-linux-x86_64.egg/gensim/test/test_data/wordsim353.tsv: 0.1016
2017-12-19 17:52:08,202 : INFO : Spearman rank-order correlation coefficient against /usr/local/lib/python2.7/dist-packages/gensim-3.2.0-py2.7-linux-x86_64.egg/gensim/test/test_data/wordsim353.tsv: 0.0829
2017-12-19 17:52:08,204 : INFO : Pairs with unknown words ratio: 85.6%


((0.10160691441325205, 0.47803354077719229),
 SpearmanrResult(correlation=0.082860052232988618, pvalue=0.56322041285528901),
 85.55240793201133)

Once again, **good performance on Google's or WS-353 test set doesn’t mean word2vec will work well in your application, or vice versa**. It’s always best to evaluate directly on your intended task. For an example of how to use word2vec in a classifier pipeline, see this [tutorial](https://github.com/RaRe-Technologies/movie-plots-by-genre).

## Storing and loading models
You can store/load models using the standard gensim methods:

In [17]:
from tempfile import mkstemp

fs, temp_path = mkstemp("gensim_temp")  # creates a temp file

model.save(temp_path)  # save the model

2017-12-19 17:52:08,220 : INFO : saving Word2Vec object under /tmp/tmp3Fmm6zgensim_temp, separately None
2017-12-19 17:52:08,223 : INFO : not storing attribute syn0norm
2017-12-19 17:52:08,223 : INFO : not storing attribute cum_table
2017-12-19 17:52:08,236 : INFO : saved /tmp/tmp3Fmm6zgensim_temp


In [18]:
new_model = gensim.models.Word2Vec.load(temp_path)  # open the model

2017-12-19 17:52:08,240 : INFO : loading Word2Vec object from /tmp/tmp3Fmm6zgensim_temp
2017-12-19 17:52:08,246 : INFO : loading wv recursively from /tmp/tmp3Fmm6zgensim_temp.wv.* with mmap=None
2017-12-19 17:52:08,247 : INFO : setting ignored attribute syn0norm to None
2017-12-19 17:52:08,247 : INFO : setting ignored attribute cum_table to None
2017-12-19 17:52:08,248 : INFO : loaded /tmp/tmp3Fmm6zgensim_temp


which uses pickle internally, optionally `mmap`‘ing the model’s internal large NumPy matrices into virtual memory directly from disk files, for inter-process memory sharing.

In addition, you can load models created by the original C tool, both using its text and binary formats:
```
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.txt', binary=False)
  # using gzipped/bz2 input works too, no need to unzip:
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.bin.gz', binary=True)
```

## Online training / Resuming training
Advanced users can load a model and continue training it with more sentences and [new vocabulary words](online_w2v_tutorial.ipynb):

In [19]:
model = gensim.models.Word2Vec.load(temp_path)
more_sentences = [['Advanced', 'users', 'can', 'load', 'a', 'model', 'and', 'continue', 'training', 'it', 'with', 'more', 'sentences']]
model.build_vocab(more_sentences, update=True)
model.train(more_sentences, total_examples=model.corpus_count, epochs=model.iter)

# cleaning up temp
os.close(fs)
os.remove(temp_path)

2017-12-19 17:52:08,258 : INFO : loading Word2Vec object from /tmp/tmp3Fmm6zgensim_temp
2017-12-19 17:52:08,264 : INFO : loading wv recursively from /tmp/tmp3Fmm6zgensim_temp.wv.* with mmap=None
2017-12-19 17:52:08,265 : INFO : setting ignored attribute syn0norm to None
2017-12-19 17:52:08,266 : INFO : setting ignored attribute cum_table to None
2017-12-19 17:52:08,266 : INFO : loaded /tmp/tmp3Fmm6zgensim_temp
2017-12-19 17:52:08,271 : INFO : collecting all words and their counts
2017-12-19 17:52:08,271 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-19 17:52:08,272 : INFO : collected 13 word types from a corpus of 13 raw words and 1 sentences
2017-12-19 17:52:08,272 : INFO : Updating model with new vocabulary
2017-12-19 17:52:08,273 : INFO : New added 0 unique words (0% of original 13) and increased the count of 0 pre-existing words (0% of original 13)
2017-12-19 17:52:08,273 : INFO : deleting the raw counts dictionary of 13 items
2017-12-19 17:52:08

You may need to tweak the `total_words` parameter to `train()`, depending on what learning rate decay you want to simulate.

Note that it’s not possible to resume training with models generated by the C tool, `KeyedVectors.load_word2vec_format()`. You can still use them for querying/similarity, but information vital for training (the vocab tree) is missing there.

## Using the model
`Word2Vec` supports several word similarity tasks out of the box:

In [20]:
model.most_similar(positive=['human', 'crime'], negative=['party'], topn=1)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2017-12-19 17:52:08,289 : INFO : precomputing L2-norms of word weight vectors


[('longer', 0.9901083707809448)]

In [21]:
model.doesnt_match("input is lunch he sentence cat".split())

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
2017-12-19 17:52:08,299 : WARNING : vectors for words set(['lunch', 'input', 'cat']) are not present in the model, ignoring these words


'sentence'

In [22]:
print(model.similarity('human', 'party'))
print(model.similarity('tree', 'murder'))

0.999207180781
0.995506030126


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


You can get the probability distribution for the center word given the context words as input:

In [23]:
print(model.predict_output_word(['emergency', 'beacon', 'received']))

[('more', 0.0010476575), ('continue', 0.00091407861), ('can', 0.0008993838), ('training', 0.00088255142), ('it', 0.00076750276), ('australia', 0.00075592351), ('there', 0.00074262387), ('could', 0.00073777919), ('or', 0.00073338969), ('which', 0.00073328678)]


The results here don't look good because the training corpus is very small. To get meaningful results one needs to train on 500k+ words.

If you need the raw output vectors in your application, you can access these either on a word-by-word basis:

In [24]:
model['tree']  # raw NumPy vector of a word

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.00667319,  0.02240003, -0.02821624, -0.00195638, -0.03252663,
       -0.03634753,  0.01395547, -0.09243303, -0.01979843, -0.02785961,
        0.02810719, -0.0192453 , -0.0654271 , -0.01889815, -0.01939129,
        0.04507412, -0.03254715,  0.00107034,  0.02099092, -0.02853582,
       -0.07192255, -0.00532108,  0.06820007,  0.04883457,  0.01651307,
        0.02314343, -0.01205105,  0.01732391, -0.0238515 , -0.00896147,
       -0.00098032,  0.06890224,  0.00935661, -0.00999498,  0.0349995 ,
        0.02084912, -0.05495631,  0.01464141,  0.00810294, -0.06084112,
       -0.08123555,  0.01204256, -0.03747954, -0.0300667 ,  0.02757824,
       -0.05517297,  0.03925103, -0.00146865, -0.01849323,  0.03568978,
        0.025634  ,  0.00453765, -0.03697179,  0.00249616,  0.01047032,
       -0.02252927,  0.05656527, -0.02479618,  0.00859999, -0.03645766,
       -0.01967132, -0.04699609,  0.02325002, -0.01824303, -0.01137256,
       -0.06449074, -0.06765109,  0.03982421,  0.06353555,  0.00

…or en-masse as a 2D NumPy matrix from `model.wv.syn0`.

## Training Loss Computation

The parameter `compute_loss` can be used to toggle computation of loss while training the Word2Vec model. The computed loss is stored in the model attribute `running_training_loss` and can be retrieved using the function `get_latest_training_loss` as follows : 

In [25]:
# instantiating and training the Word2Vec model
model_with_loss = gensim.models.Word2Vec(sentences, min_count=1, compute_loss=True, hs=0, sg=1, seed=42)

# getting the training loss value
training_loss = model_with_loss.get_latest_training_loss()
print(training_loss)

2017-12-19 17:52:08,334 : INFO : collecting all words and their counts
2017-12-19 17:52:08,337 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-19 17:52:08,362 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-12-19 17:52:08,363 : INFO : Loading a fresh vocabulary
2017-12-19 17:52:08,392 : INFO : min_count=1 retains 10186 unique words (100% of original 10186, drops 0)
2017-12-19 17:52:08,393 : INFO : min_count=1 leaves 59890 word corpus (100% of original 59890, drops 0)
2017-12-19 17:52:08,427 : INFO : deleting the raw counts dictionary of 10186 items
2017-12-19 17:52:08,428 : INFO : sample=0.001 downsamples 37 most-common words
2017-12-19 17:52:08,429 : INFO : downsampling leaves estimated 47231 word corpus (78.9% of prior 59890)
2017-12-19 17:52:08,429 : INFO : estimated required memory for 10186 words and 100 dimensions: 13241800 bytes
2017-12-19 17:52:08,454 : INFO : resetting layer weights
2017-12-19 17:52

1643973.5


### Benchmarks to see effect of training loss compuation code on training time

We first download and setup the test data used for getting the benchmarks.

In [ ]:
input_data_files = []

def setup_input_data():
    # check if test data already present
    if os.path.isfile('./text8') is False:

        # download and decompress 'text8' corpus
        import zipfile
        ! wget 'http://mattmahoney.net/dc/text8.zip'
        ! unzip 'text8.zip'
    
        # create 1 MB, 10 MB and 50 MB files
        ! head -c1000000 text8 > text8_1000000
        ! head -c10000000 text8 > text8_10000000
        ! head -c50000000 text8 > text8_50000000
                
    # add 25 KB test file
    input_data_files.append(os.path.join(os.getcwd(), '../../gensim/test/test_data/lee_background.cor'))

    # add 1 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8_1000000'))

    # add 10 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8_10000000'))

    # add 50 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8_50000000'))

    # add 100 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8'))

setup_input_data()
print(input_data_files)

--2017-12-19 17:52:09--  http://mattmahoney.net/dc/text8.zip
Resolving mattmahoney.net (mattmahoney.net)... 67.195.197.75
Connecting to mattmahoney.net (mattmahoney.net)|67.195.197.75|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31344016 (30M) [application/zip]
Saving to: ‘text8.zip’

text8.zip           100%[===================>]  29.89M   284KB/s    in 1m 54s  

2017-12-19 17:54:04 (268 KB/s) - ‘text8.zip’ saved [31344016/31344016]

Archive:  text8.zip
  inflating: text8                   
['/home/markroxor/Documents/gensim/docs/notebooks/../../gensim/test/test_data/lee_background.cor', '/home/markroxor/Documents/gensim/docs/notebooks/text8_1000000', '/home/markroxor/Documents/gensim/docs/notebooks/text8_10000000', '/home/markroxor/Documents/gensim/docs/notebooks/text8_50000000', '/home/markroxor/Documents/gensim/docs/notebooks/text8']


We now compare the training time taken for different combinations of input data and model training parameters like `hs` and `sg`.

In [ ]:
# using 25 KB and 50 MB files only for generating o/p -> comment next line for using all 5 test files
input_data_files = [input_data_files[0], input_data_files[-2]]
print(input_data_files)

import time
import numpy as np
import pandas as pd

train_time_values = []
seed_val = 42
sg_values = [0, 1]
hs_values = [0, 1]

for data_file in input_data_files:
    data = gensim.models.word2vec.LineSentence(data_file) 
    for sg_val in sg_values:
        for hs_val in hs_values:
            for loss_flag in [True, False]:
                time_taken_list = []
                for i in range(3):
                    start_time = time.time()
                    w2v_model = gensim.models.Word2Vec(data, compute_loss=loss_flag, sg=sg_val, hs=hs_val, seed=seed_val) 
                    time_taken_list.append(time.time() - start_time)

                time_taken_list = np.array(time_taken_list)
                time_mean = np.mean(time_taken_list)
                time_std = np.std(time_taken_list)
                train_time_values.append({'train_data': data_file, 'compute_loss': loss_flag, 'sg': sg_val, 'hs': hs_val, 'mean': time_mean, 'std': time_std})

train_times_table = pd.DataFrame(train_time_values)
train_times_table = train_times_table.sort_values(by=['train_data', 'sg', 'hs', 'compute_loss'], ascending=[False, False, True, False])
print(train_times_table)

['/home/markroxor/Documents/gensim/docs/notebooks/../../gensim/test/test_data/lee_background.cor', '/home/markroxor/Documents/gensim/docs/notebooks/text8_50000000']


2017-12-19 17:54:05,998 : INFO : collecting all words and their counts
2017-12-19 17:54:06,000 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-19 17:54:06,023 : INFO : collected 10781 word types from a corpus of 59890 raw words and 300 sentences
2017-12-19 17:54:06,024 : INFO : Loading a fresh vocabulary
2017-12-19 17:54:06,032 : INFO : min_count=5 retains 1762 unique words (16% of original 10781, drops 9019)
2017-12-19 17:54:06,034 : INFO : min_count=5 leaves 46084 word corpus (76% of original 59890, drops 13806)
2017-12-19 17:54:06,041 : INFO : deleting the raw counts dictionary of 10781 items
2017-12-19 17:54:06,043 : INFO : sample=0.001 downsamples 45 most-common words
2017-12-19 17:54:06,044 : INFO : downsampling leaves estimated 32610 word corpus (70.8% of prior 46084)
2017-12-19 17:54:06,045 : INFO : estimated required memory for 1762 words and 100 dimensions: 2290600 bytes
2017-12-19 17:54:06,051 : INFO : resetting layer weights
2017-12-19 17:

2017-12-19 17:54:07,398 : INFO : deleting the raw counts dictionary of 10781 items
2017-12-19 17:54:07,399 : INFO : sample=0.001 downsamples 45 most-common words
2017-12-19 17:54:07,400 : INFO : downsampling leaves estimated 32610 word corpus (70.8% of prior 46084)
2017-12-19 17:54:07,401 : INFO : estimated required memory for 1762 words and 100 dimensions: 2290600 bytes
2017-12-19 17:54:07,407 : INFO : resetting layer weights
2017-12-19 17:54:07,433 : INFO : training model with 3 workers on 1762 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2017-12-19 17:54:07,609 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-12-19 17:54:07,610 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-12-19 17:54:07,613 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-12-19 17:54:07,615 : INFO : training on 299450 raw words (162889 effective words) took 0.2s, 909238 effective words/s
2017-12-19 17:54:07,617 

2017-12-19 17:54:09,273 : INFO : deleting the raw counts dictionary of 10781 items
2017-12-19 17:54:09,275 : INFO : sample=0.001 downsamples 45 most-common words
2017-12-19 17:54:09,276 : INFO : downsampling leaves estimated 32610 word corpus (70.8% of prior 46084)
2017-12-19 17:54:09,277 : INFO : estimated required memory for 1762 words and 100 dimensions: 3347800 bytes
2017-12-19 17:54:09,280 : INFO : constructing a huffman tree from 1762 words
2017-12-19 17:54:09,323 : INFO : built huffman tree with maximum node depth 13
2017-12-19 17:54:09,328 : INFO : resetting layer weights
2017-12-19 17:54:09,347 : INFO : training model with 3 workers on 1762 vocabulary and 100 features, using sg=0 hs=1 sample=0.001 negative=5 window=5
2017-12-19 17:54:09,604 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-12-19 17:54:09,605 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-12-19 17:54:09,607 : INFO : worker thread finished; awaiting finish of 0 more

2017-12-19 17:54:11,608 : INFO : estimated required memory for 1762 words and 100 dimensions: 2290600 bytes
2017-12-19 17:54:11,614 : INFO : resetting layer weights
2017-12-19 17:54:11,634 : INFO : training model with 3 workers on 1762 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5
2017-12-19 17:54:12,066 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-12-19 17:54:12,068 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-12-19 17:54:12,077 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-12-19 17:54:12,077 : INFO : training on 299450 raw words (162889 effective words) took 0.4s, 368472 effective words/s
2017-12-19 17:54:12,079 : INFO : collecting all words and their counts
2017-12-19 17:54:12,080 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-19 17:54:12,103 : INFO : collected 10781 word types from a corpus of 59890 raw words and 300 sentences
2017-12-1

...

2017-12-19 18:26:14,297 : INFO : PROGRESS: at 74.75% examples, 170802 words/s, in_qsize 6, out_qsize 0
2017-12-19 18:26:15,308 : INFO : PROGRESS: at 75.36% examples, 170922 words/s, in_qsize 5, out_qsize 0
2017-12-19 18:26:16,375 : INFO : PROGRESS: at 75.95% examples, 170929 words/s, in_qsize 6, out_qsize 0
2017-12-19 18:26:17,420 : INFO : PROGRESS: at 76.51% examples, 170900 words/s, in_qsize 6, out_qsize 1
2017-12-19 18:26:18,519 : INFO : PROGRESS: at 77.15% examples, 170983 words/s, in_qsize 5, out_qsize 0
2017-12-19 18:26:19,533 : INFO : PROGRESS: at 77.76% examples, 171090 words/s, in_qsize 6, out_qsize 0
2017-12-19 18:26:20,558 : INFO : PROGRESS: at 78.28% examples, 170998 words/s, in_qsize 5, out_qsize 0
2017-12-19 18:26:21,578 : INFO : PROGRESS: at 78.87% examples, 171059 words/s, in_qsize 6, out_qsize 0
2017-12-19 18:26:22,628 : INFO : PROGRESS: at 79.43% examples, 171032 words/s, in_qsize 6, out_qsize 0
2017-12-19 18:26:24,011 : INFO : PROGRESS: at 79.84% examples, 170267 wor

2017-12-19 18:27:25,941 : INFO : PROGRESS: at 11.24% examples, 167336 words/s, in_qsize 6, out_qsize 1
2017-12-19 18:27:27,004 : INFO : PROGRESS: at 11.80% examples, 167281 words/s, in_qsize 6, out_qsize 0
2017-12-19 18:27:28,030 : INFO : PROGRESS: at 12.37% examples, 167546 words/s, in_qsize 6, out_qsize 0
2017-12-19 18:27:29,063 : INFO : PROGRESS: at 12.91% examples, 167401 words/s, in_qsize 6, out_qsize 1
2017-12-19 18:27:30,072 : INFO : PROGRESS: at 13.45% examples, 167452 words/s, in_qsize 6, out_qsize 0
2017-12-19 18:27:31,100 : INFO : PROGRESS: at 13.99% examples, 167366 words/s, in_qsize 5, out_qsize 0
2017-12-19 18:27:32,102 : INFO : PROGRESS: at 14.58% examples, 167996 words/s, in_qsize 5, out_qsize 0
2017-12-19 18:27:33,107 : INFO : PROGRESS: at 15.17% examples, 168469 words/s, in_qsize 5, out_qsize 0
2017-12-19 18:27:34,108 : INFO : PROGRESS: at 15.71% examples, 168430 words/s, in_qsize 6, out_qsize 0
2017-12-19 18:27:35,108 : INFO : PROGRESS: at 16.28% examples, 168646 wor

2017-12-19 18:28:48,626 : INFO : PROGRESS: at 56.40% examples, 169092 words/s, in_qsize 6, out_qsize 0
2017-12-19 18:28:49,635 : INFO : PROGRESS: at 56.96% examples, 169148 words/s, in_qsize 6, out_qsize 0
2017-12-19 18:28:50,680 : INFO : PROGRESS: at 57.53% examples, 169137 words/s, in_qsize 6, out_qsize 0
2017-12-19 18:28:51,715 : INFO : PROGRESS: at 58.07% examples, 169076 words/s, in_qsize 6, out_qsize 1
2017-12-19 18:28:52,733 : INFO : PROGRESS: at 58.66% examples, 169183 words/s, in_qsize 5, out_qsize 0
2017-12-19 18:28:53,751 : INFO : PROGRESS: at 59.20% examples, 169145 words/s, in_qsize 5, out_qsize 0
2017-12-19 18:28:54,793 : INFO : PROGRESS: at 59.76% examples, 169140 words/s, in_qsize 5, out_qsize 0
2017-12-19 18:28:55,839 : INFO : PROGRESS: at 60.00% examples, 168172 words/s, in_qsize 6, out_qsize 0
2017-12-19 18:28:56,843 : INFO : PROGRESS: at 60.57% examples, 168242 words/s, in_qsize 6, out_qsize 0
2017-12-19 18:28:57,880 : INFO : PROGRESS: at 61.15% examples, 168325 wor

2017-12-19 18:30:07,512 : INFO : training on 42426485 raw words (31022350 effective words) took 182.3s, 170129 effective words/s
2017-12-19 18:30:07,563 : INFO : collecting all words and their counts
2017-12-19 18:30:08,106 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-19 18:30:09,833 : INFO : collected 171140 word types from a corpus of 8485297 raw words and 849 sentences
2017-12-19 18:30:09,834 : INFO : Loading a fresh vocabulary
2017-12-19 18:30:10,091 : INFO : min_count=5 retains 48753 unique words (28% of original 171140, drops 122387)
2017-12-19 18:30:10,092 : INFO : min_count=5 leaves 8292974 word corpus (97% of original 8485297, drops 192323)
2017-12-19 18:30:10,260 : INFO : deleting the raw counts dictionary of 171140 items
2017-12-19 18:30:10,293 : INFO : sample=0.001 downsamples 38 most-common words
2017-12-19 18:30:10,294 : INFO : downsampling leaves estimated 6205111 word corpus (74.8% of prior 8292974)
2017-12-19 18:30:10,295 : INFO : 

2017-12-19 18:31:21,254 : INFO : PROGRESS: at 37.79% examples, 170041 words/s, in_qsize 5, out_qsize 0
2017-12-19 18:31:22,257 : INFO : PROGRESS: at 38.35% examples, 170121 words/s, in_qsize 6, out_qsize 0
2017-12-19 18:31:23,274 : INFO : PROGRESS: at 38.92% examples, 170179 words/s, in_qsize 5, out_qsize 0
2017-12-19 18:31:24,289 : INFO : PROGRESS: at 39.51% examples, 170309 words/s, in_qsize 6, out_qsize 0
2017-12-19 18:31:25,589 : INFO : PROGRESS: at 39.86% examples, 168805 words/s, in_qsize 4, out_qsize 2
2017-12-19 18:31:26,631 : INFO : PROGRESS: at 40.42% examples, 168757 words/s, in_qsize 6, out_qsize 0
2017-12-19 18:31:27,643 : INFO : PROGRESS: at 40.97% examples, 168720 words/s, in_qsize 5, out_qsize 0
2017-12-19 18:31:28,646 : INFO : PROGRESS: at 41.58% examples, 168973 words/s, in_qsize 6, out_qsize 0
2017-12-19 18:31:29,688 : INFO : PROGRESS: at 42.17% examples, 169047 words/s, in_qsize 5, out_qsize 0
2017-12-19 18:31:30,686 : INFO : PROGRESS: at 42.76% examples, 169121 wor

### Adding Word2Vec "model to dict" method to production pipeline
Suppose, we still want more performance improvement in production. 
One good way is to cache all the similar words in a dictionary.
So that next time when we get the similar query word, we'll search it first in the dict.
And if it's a hit then we will show the result directly from the dictionary.
otherwise we will query the word and then cache it so that it doesn't miss next time.

In [ ]:
most_similars_precalc = {word : model.wv.most_similar(word) for word in model.wv.index2word}
print(most_similars_precalc)

### Comparison with and without caching

for time being lets take 4 words randomly

In [ ]:
import time
words = ['voted','few','their','around']

Without caching

In [ ]:
start = time.time()
for word in words:
    result = model.wv.most_similar(word)
    print(result)
end = time.time()
print(end-start)

Now with caching

In [ ]:
start = time.time()
for word in words:
    if 'voted' in most_similars_precalc:
        result = most_similars_precalc[word]
        print(result)
    else:
        result = model.wv.most_similar(word)
        most_similars_precalc[word] = result
        print(result)
    
end = time.time()
print(end-start)

Clearly you can see the improvement but this difference will be even larger when we take more words in the consideration.

# Visualising the Word Embeddings

The word embeddings made by the model can be visualised by reducing dimensionality of the words to 2 dimensions using tSNE.

Visualisations can be used to notice semantic and syntactic trends in the data.

Example: Semantic- words like cat, dog, cow, etc. have a tendency to lie close by
         Syntactic- words like run, running or cut, cutting lie close together.
Vector relations like vKing - vMan = vQueen - vWoman can also be noticed.

Additional dependencies : 
- sklearn
- numpy
- plotly

The function below can be used to plot the embeddings in an ipython notebook.
It requires the model as the necessary parameter. If you don't have the model, you can load it by

`model = gensim.models.Word2Vec.load('path/to/model')`

If you don't want to plot inside a notebook, set the `plot_in_notebook` parameter to `False`.

Note: the model used for the visualisation is trained on a small corpus. Thus some of the relations might not be so clear

Beware : This sort dimension reduction comes at the cost of loss of information.

In [ ]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling

from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go

def reduce_dimensions(model, plot_in_notebook = True):

    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = []        # positions in vector space
    labels = []         # keep track of words to label our data again later
    for word in model.wv.vocab:
        vectors.append(model[word])
        labels.append(word)


    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)
    
    # reduce using t-SNE
    vectors = np.asarray(vectors)
    logging.info('starting tSNE dimensionality reduction. This may take some time.')
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
        
    # Create a trace
    trace = go.Scatter(
        x=x_vals,
        y=y_vals,
        mode='text',
        text=labels
        )
    
    data = [trace]
    
    logging.info('All done. Plotting.')
    
    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')

In [ ]:
reduce_dimensions(model)

## Conclusion

In this tutorial we learned how to train word2vec models on your custom data and also how to evaluate it. Hope that you too will find this popular tool useful in your Machine Learning tasks!

## Links


Full `word2vec` API docs [here](http://radimrehurek.com/gensim/models/word2vec.html); get [gensim](http://radimrehurek.com/gensim/) here. Original C toolkit and `word2vec` papers by Google [here](https://code.google.com/archive/p/word2vec/).